### 함수

------------------------------------

## 카카오맵에서 구를 가져오는 함수

In [15]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from bs4 import BeautifulSoup 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.keys import Keys
import urllib.parse 
import re 
import time 
from selenium.common.exceptions import TimeoutException  

def get_addr(location_name):
    options = webdriver.ChromeOptions()
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
    options.add_argument('window-size=1380,900')
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    
    try:
        search_url = f"https://map.kakao.com/?{urllib.parse.quote(location_name)}"
        driver.get(search_url)
        
        #검색창 잔여내용삭제
        search_area = driver.find_element(By.ID, "search.keyword.query")
        search_area.clear()

        # 검색창 00역 카페 입력
        search_area.send_keys(location_name)
        driver.find_element(By.ID, 'search.keyword.submit').send_keys(Keys.ENTER)

        # 로딩 시간 기다리기(검색박스가 서치가 될때까지 로딩)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "box_searchbar.search_on")))
        
        # 검색 결과에서 주소 추출
    
        address_parts = driver.find_element(By.XPATH,"""//*[@id="info.search.place.list"]/li[1]/div[5]/div[2]/p[1]""").text
        address_parts = address_parts.split(' ')[1]
  
        # 주소에서 행정구역 추출 (서울은 구, 그 외는 시/군)
        if address_parts.endswith('구'):
            part = re.search(r'([가-힣]+구)', address_parts).group()
            return part
        else:
            if address_parts.endswith('시') or address_parts.endswith('군'):
                if "과천시" in address_parts:
                    part = "서초구"
                    return part
                elif "구리시" in address_parts:
                    part = "광진구"
                    return part   
                else:
                    return part  
        
    except Exception as e:
        print(f"주소 찾기 오류 ({location_name}): {str(e)}")
        return "지역 미상"


--------------

### 구글에서 근처 역가져오기

In [16]:

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import re
from urllib.parse import quote
from bs4 import BeautifulSoup


# 구글 맵에서 검색 후 첫 번째 결과 이름 가져오기
def get_first_result_name(search_query):
    options = webdriver.ChromeOptions()
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
    options.add_argument('window-size=1380,900')
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    
    try:
        driver.get("https://maps.google.com")
        search_box = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "searchboxinput"))
        )
        search_box.clear()
        search_box.send_keys(search_query)
        search_box.send_keys(Keys.ENTER)
        # 대기 시간 축소
        first_result = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, "Nv2PK"))
        )
        name_element = first_result.find_element(By.CLASS_NAME, "qBF1Pd")
        return name_element.text
    except:
        return None
    finally:
        driver.quit()

-----------------------------

----------------------

### 실행함수

In [17]:
import mysql.connector

def execute_sql(query, params=None):
    conn = None
    cursor = None
    try:
        conn = mysql.connector.connect(
            host="database-1.cnu82kme6p4d.ap-northeast-2.rds.amazonaws.com",
            port=3306,
            user="eda",
            password="ojk0707",
            database="ojk"
        )
        cursor = conn.cursor()
        cursor.execute(query, params)
        
        if query.strip().upper().startswith("SELECT"):
            return cursor.fetchall()
        else:
            conn.commit()
            return None
    except Exception as e:
        print(f"SQL 오류: {e}")
        return None
    finally:
        if cursor: 
            cursor.close()
        if conn: 
            conn.close()

------------

## DB에 저장하기

In [18]:
##### API 가져와보기
import requests
import pandas as pd 
from urllib.parse import quote
from bs4 import BeautifulSoup
import re
import mysql.connector

# 서울 지역 데이터 가져오기
for i in range(1, 117):
    area_list = []

    #api 호출
    area = f"POI{i:03d}"
    api_key = '6e51654f776b353335387168745664'
    url = f'http://openapi.seoul.go.kr:8088/{api_key}/xml/citydata/1/1000/{area}'
    
    #호출 읽기
    response = requests.get(url, timeout=10)
    xml_data = response.text
    soup = BeautifulSoup(xml_data, 'html.parser')


    # 명소명 처리
    name_region = soup.find('area_nm').text.strip()
    locations = [
        '광화문·덕수궁', '국립중앙박물관·용산가족공원', '낙산공원·이화마을', '덕수궁길·정동길', '서리풀공원·몽마르뜨공원',
        '서울식물원·마곡나루역', '신촌·이대역', '오목교역·목동운동장', '종로·청계 관광특구', '창덕궁·종묘', '해방촌·경리단길'
    ]
    if name_region in locations:
        area_nm = name_region.split('·')[-1]  # 뒷부분 선택
    else:
        area_nm = name_region


    # 지하철 역
    try:
        sub = soup.find('sub_stn_nm')
        station_raw = sub.text + '역'
    except:
        area_nm_search = area_nm.replace("·", "") + ' 근처 역'
        station_raw = get_first_result_name(area_nm_search)
        if station_raw:
            # 괄호와 내용 제거 (먼저 실행)
            station_raw = re.sub(r"\(.*?\)", "", station_raw)

            # 문제 역 교정
            if "4.19민주묘지" in station_raw:
                station_raw = "수유역"
            elif "응암순환" in station_raw:
                station_raw = "응암역"
            elif "대공원고가교" in station_raw:
                station_raw = "청계산입구역"
            elif "홍대입구역(2호선)" in station_raw:
                station_raw = "홍대입구역"
            # "역"이 포함된 경우, "역" 뒤 제거
            if "역" not in station_raw:
                station_raw = station_raw + "역"
        else:
            station_raw = "역 정보 없음"

  
    # 괄호와 괄호 안의 내용 제거 (정규 표현식 사용)
    if station_raw not in "역삼역":
        station_raw = re.sub(r"(역).*", r"\1", station_raw)
        station_raw = re.sub(r'\([^()]*\)', '', station_raw)


    # datetime
    update_time = soup.find('ppltn_time').text
    #최대 인구
    max_pepl = int(soup.find('area_ppltn_min').text)
    #최소 인구
    min_pepl = int(soup.find('area_ppltn_max').text)
    #기온
    temp = float(soup.find('temp').text)
    #강수
    rain_mm = soup.find('precipitation').text
    if rain_mm == "-":
        rain_mm = 0
    else:
        rain_mm_match = re.search(r'\d+', rain_mm)  # 숫자만 찾기
        rain_mm = int(rain_mm_match.group()) if rain_mm_match else 0
    #강수형태
    rain_type = soup.find('precpt_type').text
    #대기질
    air_type = soup.find('air_idx_mvl').text
    #하늘상태
    sky_stts = soup.find('sky_stts').text
    #습도
    humidity = int(soup.find('humidity').text)
    #풍향
    wind_level = float(soup.find('wind_spd').text)
    #자외선 지수
    uv = soup.find('uv_index_lvl').text.strip()
    uv_index = 0 if uv == '점검중' else float(uv)
    

    #구
    try:
        address_element = soup.find('address')
        if address_element:
            address = address_element.text
            match = re.search(r'[가-힣]+구', address)
            if match:
                gu = match.group()
            else:
                gu = get_addr(area_nm)
        else:
            gu = get_addr(area_nm)
    except:
        gu = get_addr(area_nm)

  
    
    '''
    [('datetime', 'datetime', 'NO', 'PRI', None, ''),
    ('name', 'varchar(32)', 'NO', 'PRI', None, ''),
    ('station', 'varchar(32)', 'YES', '', None, ''),
    ('gu', 'varchar(32)', 'YES', '', None, ''),
    ('min_population', 'int', 'YES', '', None, ''),
    ('max_population', 'int', 'YES', '', None, ''),
    ('temp', 'float', 'YES', '', None, ''),
    ('precipitation', 'int', 'YES', '', None, ''),
    ('precpt_type', 'varchar(32)', 'YES', '', None, ''),
    ('air_idx', 'float', 'YES', '', None, ''),
    ('sky_stts', 'varchar(32)', 'YES', '', None, ''),
    ('humidity', 'int', 'YES', '', None, ''),
    ('wind_spd', 'float', 'YES', '', None, ''),
    ('uv_idx', 'float', 'YES', '', None, '')]
    '''
    area_list += [update_time, area_nm, station_raw, gu, min_pepl, max_pepl, temp, rain_mm,  rain_type, air_type, sky_stts, humidity , wind_level, uv_index]

    query = """INSERT INTO spot (datetime, name, station, gu, min_population, max_population, temp, precipitation,
    precpt_type, air_idx, sky_stts, humidity, wind_spd, uv_idx) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    execute_sql(query, area_list)

    print(update_time,"/", area_nm,"/", station_raw,"/", gu,"/", min_pepl,"/", max_pepl,"/", temp,"/", rain_mm,"/", rain_type, "/",air_type,"/", sky_stts,"/", humidity, "/",wind_level, "/",uv_index)

2025-04-07 23:00 / 강남 MICE 관광특구 / 삼성역 / 강남구 / 2000 / 1500 / 15.0 / 0 / 없음 / 91.0 / 맑음 / 46 / 2.0 / 0.0
2025-04-07 23:00 / 동대문 관광특구 / 동대문역 / 중구 / 22000 / 20000 / 15.0 / 0 / 없음 / 88.0 / 맑음 / 50 / 4.0 / 0.0
2025-04-07 23:00 / 명동 관광특구 / 시청역 / 중구 / 12000 / 10000 / 15.0 / 0 / 없음 / 87.0 / 맑음 / 50 / 4.0 / 0.0
2025-04-07 23:00 / 이태원 관광특구 / 녹사평역 / 용산구 / 5000 / 4500 / 15.6 / 0 / 없음 / 82.0 / 맑음 / 43 / 3.0 / 0.0
2025-04-07 23:00 / 잠실 관광특구 / 잠실역 / 송파구 / 28000 / 26000 / 15.8 / 0 / 없음 / 86.0 / 맑음 / 43 / 1.6 / 0.0
2025-04-07 23:00 / 청계 관광특구 / 동대문역 / 종로구 / 12000 / 10000 / 15.0 / 0 / 없음 / 88.0 / 맑음 / 50 / 4.0 / 0.0
2025-04-07 23:00 / 홍대 관광특구 / 합정역 / 마포구 / 36000 / 34000 / 15.4 / 0 / 없음 / 93.0 / 맑음 / 45 / 2.0 / 0.0
2025-04-07 23:00 / 경복궁 / 경복궁역 / 종로구 / 400 / 300 / 15.0 / 0 / 없음 / 88.0 / 맑음 / 50 / 4.0 / 0.0
2025-04-07 23:00 / 덕수궁 / 시청역 / 중구 / 5500 / 5000 / 15.0 / 0 / 없음 / 88.0 / 맑음 / 50 / 4.0 / 0.0
2025-04-07 23:00 / 보신각 / 종각역 / 종로구 / 3500 / 3000 / 15.0 / 0 / 없음 / 88.0 / 맑음 / 50 / 4.0 / 0.0
2025-04-07 23:00

In [20]:
import mysql.connector
import pandas as pd

def execute_sql(sql, params=None, as_dataframe=False):
    ojk = mysql.connector.connect(
        host = "database-1.cnu82kme6p4d.ap-northeast-2.rds.amazonaws.com",
        port = 3306,
        user = "eda",
        password = "ojk0707",
        database = "ojk"
    )
    cursor = ojk.cursor(dictionary=True if as_dataframe else False)
    
    if params:
        cursor.execute(sql, params)
        ojk.commit()
    else:
        cursor.execute(sql)
        
    results = cursor.fetchall()
    
    cursor.close()
    ojk.close()
    
    if as_dataframe:
        return pd.DataFrame(results)
    return results

In [21]:
execute_sql("select * from spot", as_dataframe=True)

,datetime,name,station,gu,min_population,max_population,temp,precipitation,precpt_type,air_idx,sky_stts,humidity,wind_spd,uv_idx
0,2025-04-07 23:00:00,4·19 카페거리,수유역,강북구,1000,900,14.8,0,없음,82.0,맑음,44,4.3,0.0
1,2025-04-07 23:00:00,가락시장,가락시장역,송파구,7500,7000,16.1,0,없음,86.0,맑음,40,3.1,0.0
2,2025-04-07 23:00:00,가로수길,신사역,강남구,10000,9500,15.6,0,없음,91.0,맑음,43,3.0,0.0
3,2025-04-07 23:00:00,가산디지털단지역,가산디지털단지역,금천구,9500,9000,14.8,0,없음,85.0,맑음,48,2.1,0.0
4,2025-04-07 23:00:00,강남 MICE 관광특구,삼성역,강남구,2000,1500,15.0,0,없음,91.0,맑음,46,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,2025-04-07 23:05:00,잠원한강공원,신사역,강남구,900,800,15.6,0,없음,91.0,맑음,43,3.0,0.0
112,2025-04-07 23:05:00,청계산,청계산입구역,서초구,1500,1000,14.8,0,없음,90.0,맑음,45,3.3,0.0
113,2025-04-07 23:05:00,청담동 명품거리,압구정로데오역,강남구,2500,2000,15.6,0,없음,91.0,맑음,43,3.0,0.0
114,2025-04-07 23:05:00,청량리 제기동 일대 전통시장,제기동역,동대문구,10000,9500,16.1,0,없음,86.0,맑음,40,2.9,0.0


In [13]:
execute_sql("truncate table spot")

[]